In [19]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from SyntheticControlMethods import Synth
import synth_utils as su
import pandas as pd
import numpy as np
import synth_job as sj
import Dino.cf_value as cfv
import Dino.counterfeit_utils as cfu
from scipy.stats import percentileofscore

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
df_1 = pd.read_pickle('sales.pkl')
df_2 = pd.read_pickle('master_df.pkl')
df_1['date'] = pd.to_datetime(cfu.timestamps_to_dates(df_1['timestamp'],raw_timestamps=True))
#cast alt_date as a datetime in a row called date
df_2['date'] = pd.to_datetime(df_2['alt_date'])


In [34]:
import pandas as pd
import statsmodels.api as sm
from tqdm import tqdm
#iterate through rows of master df
full_results_list = []
for index, row in tqdm(df_2.iterrows()):
    #get all rows in sales with dates plus or minus 1 day of the date in the master df
    df_1_subset = df_1[(df_1['date'] >= row['date'] - pd.DateOffset(days=1)) & (df_1['date'] <= row['date'] + pd.DateOffset(days=1))]
    results_list = []
    for slug, slug_df in df_1_subset.groupby('slug'):
        # Sort the dates for this slug
        unique_dates = sorted(slug_df['date'].unique())
        
        # We need at least 3 unique dates to do (first) vs. (second+third)
        if len(unique_dates) < 3:
            continue
        
        # Identify the first, second, third dates
        first_date = unique_dates[0]
        second_date = unique_dates[1]
        third_date  = unique_dates[2]
        
        # Subset: train on first_date, validate on [second_date, third_date]
        train_df = slug_df[slug_df['date'] == first_date]
        val_df   = slug_df[slug_df['date'].isin([second_date, third_date])]
        
        # Skip if fewer than 5 rows in the training or validation sets
        if (train_df['timestamp'].nunique() < 5) or (val_df['timestamp'].nunique() < 5):
            continue
        # Fit linear model: price ~ timestamp, using statsmodels
        X_train = sm.add_constant(train_df['timestamp'])
        y_train = train_df['price']
        model   = sm.OLS(y_train, X_train).fit()
        
        # Evaluate on combined validation set
        X_val = sm.add_constant(val_df['timestamp'], has_constant='add')
        y_val = val_df['price']
        y_pred = model.predict(X_val)
        
        # For demonstration, compute MSE
        mse_pred = ((y_val - y_pred) ** 2).mean()
        mse_train = ((y_train - model.predict(X_train)) ** 2).mean()
        # ss_res = ((y_val - y_pred) ** 2).sum()                        # Residual Sum of Squares
        # ss_tot = ((y_val - y_val.mean()) ** 2).sum()                  # Total Sum of Squares
        # r2_val = 1 - ss_res / ss_tot if ss_tot != 0 else float('nan')
        results_list.append({
            'slug':        slug,
            'first_date':  first_date.date(),
            'second_date': second_date.date(),
            'third_date':  third_date.date(),
            'n_train':     len(train_df),
            'n_val':       len(val_df),
            'mse_val':     mse_pred/mse_train,
            # 'r2_val':        r2_val,
        })

    results_df = pd.DataFrame(results_list)
    if row['Top_100'] not in results_df['slug'].values:
        print(f"Slug {row['Top_100']} not found in results_df")
        continue
    my_mse = results_df.loc[results_df['slug'] == row['Top_100'], 'mse_val'].iloc[0]

    # 2) Compute quantile or percentile using percentileofscore 
    quantile_percent_mse = percentileofscore(results_df['mse_val'], my_mse, kind='rank')
    # quantile_percent_r2 = percentileofscore(results_df['r2_val'], r2_val, kind='rank')

    # print(f"MSE for slug {row['Top_100']} = {my_mse}")
    # print(f"It is at the {quantile_percent_mse:.3f}th percentile among all slugs in mse.")
    full_results_list.append((row['Top_100'],row['Alt'], my_mse, quantile_percent_mse, len(results_df)))
full_results_df = pd.DataFrame(full_results_list, columns=['slug', 'alt', 'mse', 'percentile','num_samples'])

0it [00:00, ?it/s]

2it [00:01,  1.64it/s]/tmp/ipykernel_68553/1115896522.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  'mse_val':     mse_pred/mse_train,
3it [00:01,  1.58it/s]

Slug 0n1-force not found in results_df


13it [00:07,  1.95it/s]

Slug 3landers not found in results_df


...

1448it [14:20,  1.54it/s]

Slug world-of-women-galaxy not found in results_df


1454it [14:24,  1.48it/s]

Slug world-of-women-nft not found in results_df


1458it [14:27,  1.68it/s]


In [43]:
import math
def hoeffding_confidence_interval(data, delta=0.05):
    """
    Returns a (lower, upper) confidence interval for the mean of 'data'
    using Hoeffding's inequality at confidence level 1 - delta.
    """
    data = np.array(data)/100
    n = len(data)
    sample_mean = np.mean(data)
    
    # Hoeffding bound for [0,1] random variables:
    #   P(|sample_mean - true_mean| >= epsilon) <= 2*exp(-2*n*epsilon^2)
    # Solve for epsilon when the RHS = delta:
    #   2*exp(-2*n*epsilon^2) = delta  ==>  epsilon = sqrt(ln(2/delta)/(2*n))
    epsilon = math.sqrt( math.log(2.0/delta) / (2.0 * n) )
    
    lower = sample_mean - epsilon
    upper = sample_mean + epsilon
    
    # You might optionally clip the interval to [0,1] if you know the true mean must be in [0,1].
    lower = max(0, lower)
    upper = min(1, upper)

    return lower, upper

In [44]:
for slug, slug_df in full_results_df.groupby('slug'):
    mse_list = slug_df['percentile'].tolist()
    lower, upper = hoeffding_confidence_interval(mse_list)
    print(f"Slug {slug}: Hoeffding CI = ({lower:.3f}, {upper:.3f})")

Slug 0n1-force: Hoeffding CI = (0.000, 0.776)
Slug 3landers: Hoeffding CI = (0.201, 0.689)
Slug alienfrensnft: Hoeffding CI = (0.064, 0.630)
Slug azuki: Hoeffding CI = (0.000, 1.000)
Slug beanzofficial: Hoeffding CI = (0.000, 0.661)
Slug bears-deluxe: Hoeffding CI = (0.000, 1.000)
Slug boredapeyachtclub: Hoeffding CI = (0.000, 1.000)
Slug chain-runners-nft: Hoeffding CI = (0.000, 1.000)
Slug clonex: Hoeffding CI = (0.033, 0.692)
Slug cool-cats-nft: Hoeffding CI = (0.269, 0.632)
Slug coolmans-universe: Hoeffding CI = (0.096, 0.822)
Slug cryptodickbutts-s3: Hoeffding CI = (0.000, 0.938)
Slug cryptopunks: Hoeffding CI = (0.290, 0.794)
Slug doodles-official: Hoeffding CI = (0.000, 1.000)
Slug forgottenruneswizardscult: Hoeffding CI = (0.000, 1.000)
Slug genuine-undead: Hoeffding CI = (0.000, 0.736)
Slug goblintownwtf: Hoeffding CI = (0.000, 0.886)
Slug guttercatgang: Hoeffding CI = (0.000, 1.000)
Slug hashmasks: Hoeffding CI = (0.000, 1.000)
Slug kaiju-kingz: Hoeffding CI = (0.000, 0.695)


In [40]:
full_results_df

,slug,alt,mse,percentile,num_samples
0,0n1-force,0xvampire-project,22.037226,82.658960,173
1,0n1-force,0xzuki,1.294028,22.072072,444
2,0n1-force,roars-by-ntp,2.583635,26.219512,164
3,0n1-force,mekazuki,6.317140,48.306998,443
4,0n1-force,ryokoclub,2.357950,24.429967,307
...,...,...,...,...,...
982,world-of-women-nft,stardustgeneration,31.912046,74.571429,350
983,world-of-women-nft,asian-identity-collection,0.469795,5.116279,430
984,world-of-women-nft,world-of-boys-official,6.299854,53.571429,420
985,world-of-women-nft,stargals,11.199422,57.812500,320
